In [51]:
import torch
from torch import nn
from torch import optim

torch.manual_seed(777)
if torch.cuda.is_available():
    device = 'cuda'
    torch.cuda.manual_seed_all(777)
else:
    device = 'cpu'

In [52]:
x = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

INPUTS = 2
HIDDEN_INPUTS = 2
OUTPUTS = 1


class GateModel(nn.Module):
    def __init__(self, i, h, o=1, l=1):
        super(GateModel, self).__init__()

        print(f"Model : Inputs = {i} | Hidden Outputs = {h} | Output = {o} | Layers = {l}")
        self.layer = nn.Sequential()
        if not l or l == 1:
            self.layer.append(nn.Linear(i, o, bias=True))
            self.layer.append(nn.Sigmoid())
        elif l and l > 1:
            self.layer.append(nn.Linear(i, h, bias=True))
            self.layer.append(nn.Sigmoid())
            for k in range(l - 1):
                self.layer.append(nn.Linear(h, h, bias=True))
                self.layer.append(nn.Sigmoid())
            self.layer.append(nn.Linear(h, o, bias=True))
            self.layer.append(nn.Sigmoid())

    def forward(self, x):
        return self.layer(x)


def train(x, y, model, criterion, optimizer, epochs=10000):
    #[l.reset_parameters() for l in model.layer if isinstance(l, nn.Conv2d) or isinstance(l, nn.Linear)]

    priod = round(epochs / 10)
    print(f"\nStart ... for {epochs}")
    for epoch in range(epochs):
        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % priod == 0: print(f"Epoch : {epoch + 1:4d}, Cost : {loss:.3f}")
    print("\nFinal Cost = ", loss)

    with torch.no_grad():
        model.eval()
        print("x =", x)
        print("o = ", model(x) > 0.5)
        print("y =", y)


model = GateModel(2, 2, l=2).to(device)
criterion = nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

train(x, y, model, criterion, optimizer)

Model : Inputs = 2 | Hidden Outputs = 2 | Output = 1 | Layers = 2

Start ... for 10000
Epoch : 1000, Cost : 0.693
Epoch : 2000, Cost : 0.693
Epoch : 3000, Cost : 0.693
Epoch : 4000, Cost : 0.486
Epoch : 5000, Cost : 0.018
Epoch : 6000, Cost : 0.006
Epoch : 7000, Cost : 0.004
Epoch : 8000, Cost : 0.003
Epoch : 9000, Cost : 0.002
Epoch : 10000, Cost : 0.002

Final Cost =  tensor(0.0016, grad_fn=<BinaryCrossEntropyBackward0>)
x = tensor([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]])
o =  tensor([[False],
        [ True],
        [ True],
        [False]])
y = tensor([[0.],
        [1.],
        [1.],
        [0.]])
